In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [51]:
import os, getpass
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]
REVIEW_INDEX = 'reviews'

llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [11]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool
from langchain.tools import BaseTool

In [13]:
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="structure_content",
        func=search.run,
        description="useful for when you need to structure course content into different headings and subcompotents to help students create a holistic overview of course material and concepts"
    )
]

In [16]:
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

life_tool = Tool(
    name="Meaning of Life",
    func=meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL"
    )

In [17]:
import random

def random_num(input=""):
    return random.randint(0,5)

In [18]:
random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

In [19]:
from langchain.agents import initialize_agent

tools = [search, random_tool, life_tool]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

C:\Users\Schalk\anaconda3\envs\sandbox1\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [39]:
#conversational_agent("What time is it in London?")

In [22]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [26]:
GPLACES_API_KEY =os.environ["GPLACES_API_KEY"] 

In [30]:
from langchain_google_community import GooglePlacesTool

In [32]:
places = GooglePlacesTool()

In [38]:
places.run("Rust en Vrede")

'1. Rust en Vrede Wine Estate\nAddress: Annandale Rd, Stellenbosch, 7600, South Africa\nGoogle place ID: ChIJJejU6OKxzR0RO5WAGlKasYg\nPhone: 021 881 3881\nWebsite: http://rustenvrede.com/\n\n'

In [42]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [43]:
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]

In [44]:
look_for = TavilySearchResults(max_results=2)

In [48]:
#look_for.invoke("what are aliens?")

In [52]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'  
embeddings = OpenAIEmbeddings(  
    model=model_name,  
    openai_api_key=OPENAI_API_KEY
)

vectorstore_reviews = PineconeVectorStore(index_name=REVIEW_INDEX, embedding=embeddings)
retriever_reviews = vectorstore_reviews.as_retriever()

In [54]:
retriever_reviews.invoke("whats the worst coffee spot near me")

[Document(page_content='have any toilet, but the drink was good and they have spaces to sit.', metadata={'address': '67 Main Street, Brooklyn', 'name': 'Starbucks', 'place_id': 'ChIJrY4KOzFawokRDCaABlBimbM', 'review_time': '2024-03-16T06:32:36'}),
 Document(page_content='worst place ever and rude waitress.', metadata={'address': '146 Beekman Street, New York', 'name': 'Fresh Salt', 'place_id': 'ChIJN8ko6jxawokRh4mOVeF82dg', 'review_time': '2024-05-18T12:09:26'}),
 Document(page_content='place is overpriced for mediocre food and horrible service. We will never return.', metadata={'address': '200 Clinton Street, Brooklyn', 'name': 'Atlantic Bagels Cafe', 'place_id': 'ChIJO0J-fE9awokRvCs8n1y90eA', 'review_time': '2024-04-24T16:21:14'}),
 Document(page_content='at this location definitely do. Thank you, thank you, thank you!', metadata={'address': 'Fulton Mall, 348 Fulton Street, Brooklyn', 'name': 'Starbucks', 'place_id': 'ChIJFbie-UtawokRr6eHiB7eBoU', 'review_time': '2023-05-06T11:09:41'

In [55]:
from langchain.tools.retriever import create_retriever_tool

In [75]:
retriever_tool = create_retriever_tool(
    retriever_reviews,
    "google_reviews",
    "When making recommendations about food and drinks spots, you must use this tool to find reviews!",
)

In [81]:
tools = [look_for, retriever_tool]

In [82]:
from langchain_core.messages import HumanMessage

response = llm.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [83]:
model_with_tools = llm.bind_tools(tools)

In [85]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [87]:
response = model_with_tools.invoke([HumanMessage(content="Where should i get coffee?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'google_reviews', 'args': {'query': 'best coffee shops near me'}, 'id': 'call_Ml2xOtCQ8elQAuLplZqyerFi'}]


In [88]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [96]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt)

In [95]:
#help(create_tool_calling_agent)

In [97]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [98]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [99]:
agent_executor.invoke({"input": "where should i get coffee in brooklyn"})

{'input': 'where should i get coffee in brooklyn',
 'output': "I found some reviews for coffee shops in Brooklyn. Here are a few options based on the reviews:\n\n1. Worth Walking To\n2. Really Friendly and Welcoming\n3. Breakfast Menu the Next Time We're in Town\n4. Feels Like a Little Slice of Mayberry\n\nThese coffee shops seem to have positive reviews and could be great options for you to try in Brooklyn."}

In [102]:
agent_executor.invoke({"input": "who won the most recent NBA season"})

{'input': 'who won the most recent NBA season',
 'output': 'The most recent NBA season winner was the Milwaukee Bucks in 2021.'}

In [103]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})

{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [104]:
from langchain_core.messages import AIMessage, HumanMessage

In [105]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)

{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob! How can I assist you, Bob?'}

In [106]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [107]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [108]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [109]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob! How can I help you, Bob?'}